In [ ]:
# ==============================================================+
# Function Testing with pytest                                  |
# Author    : Toby Law                                          |                                   |
# Date      : 10/29/2021                                        |
# ==============================================================+

## Function testing with pytest
*****

### Function to be tested: `cipher`, enciphers `text` by replacing each letter by another some fixed number of positions down the alphabet.

In [1]:
def cipher(text, shift, encrypt=True):
    alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'
    new_text = ''
    for c in text:
        index = alphabet.find(c)
        if index == -1:
            new_text += c
        else:
            new_index = index + shift if encrypt == True else index - shift
            new_index %= len(alphabet) #this line conforms shift values larger than the length of alphabet
            new_text += alphabet[new_index:new_index+1]
    return new_text

### 1. Testing the `cipher` function

   ### (a) Checking whether `cipher` works on a single word

In [ ]:
def test_singleword():
    """Testing the cipher function on single words."""
    actual = cipher(text = "Hello", shift = 2)
    expected = "Jgnnq"
    assert actual == expected, "cipher has failed on a single word."

### (b) Checking whether negative _shift_ values are accepted

In [ ]:
def test_negshift():
    """Testing if negative shift values are accepted."""
    actual = cipher(text = "Hello", shift = -1)
    expected = "Gdkkn"
    assert actual == expected, "cipher did not accept negative shift values."

### (c) Check if the original text is returned if it contains symbols not within the alphabet

In [ ]:
def test_ooa():
    """Testing if out-of-alphabet values are returned as they are"""
    actual = cipher(text = "Hel123lo", shift = 2)
    expected = 'Jgn123nq'
    assert actual == expected, "Out-of-alphabet values were modified."

### (d)  Checking if submitting string _shift_ values return the correct exception

In [ ]:
def test_valueerror_on_string_shift():
    """Test if submitting a string to shift argument returns TypeError."""
    with pytest.raises(TypeError) as exception_info:
        cipher(text = "Hello", shift = 'two')

### (e) Please see `test_cipher.py` script, and screenshot below for the output of tests in parts 1(a) - (d).

<img src="test_cipher_results.JPG" width="600" height="600"/>

### (f) Use `@pytest.fixture` to define and submit multiple example test strings and the desired shift to the `test_multiple` function. The _for_ loop is used to iterate over the `test_pairs`, and assert for each case whether the string variation can be handled by `cipher`.

In [ ]:
@pytest.fixture
def test_pairs():
    return [
        ("MixeD", "PlAhG", 3),
        ("lower", "qtBjw", 5),
        ("UPPER", "Avvkx", 32),
        ("A sentence WITH spaCes", "H zluAlujl dPaO zwhJlz", 7)
    ]

def test_multiple(test_pairs):
    """Testing for multiple string variations."""
    for pair in test_pairs:
        example = pair[0]
        expected = pair[1]
        result = cipher(text = example, shift = pair[2], encrypt = True)
        assert result == expected, "cipher cannot handle all string variations."

### (g) Write a test function that calls `cipher` twice, once to encrypt and once to decrypt, and compare the outcome to the original string. Use` @pytest.mark.parametrize` to run this test for `shift` values from 1 to 10.

In [ ]:
@pytest.mark.parametrize("shift", list(range(1, 11)))

def test_integrate(shift):
    """Check that cipher can encrypt and decrypt faithfullya."""
    original = "Switching Back and Forth."
    encrypt = cipher(text = original, shift = shift, encrypt = True)
    decrypt = cipher(text = encrypt, shift = shift, encrypt = False)
    assert original == decrypt, "Decrypted text not the same as original!"

#### All of 1(a) - (g) are added to the `test_cipher.py` script, all test results are in screenshot below.

<img src="test_cipher_all_results.JPG" width="600" height="600"/>

### 2. Added test methods in 1(a) - (d) to a test class. Parametrized the class so that 5 test cases below will be sent to all test functions within this class. 

In [ ]:
test_cases = [
    ("Hello!", "two", "Jgnnq!"),
    ("MixeD", 3, "PlAhG"),
    ("lower", 5, "qtBjw"),
    ("UPPER", -32, "ojjYl"),
    ("A sentence WITH spaCes?", -7, "t lXgmXgVX PBMA liTvXl?")
]

@pytest.mark.parametrize('example, shift, result', test_cases)

class TestCipher:
    def test_singleword(self, example, shift, result):
        """Testing the cipher function on single words."""
        if isinstance(shift, int):
            actual = cipher(text = example, shift = shift)
            expected = result
            assert actual == expected, "cipher has failed on a single word."
    
    def test_negshift(self, example, shift, result):
        """Testing if negative shift values are accepted."""
        if isinstance(shift, int) and shift < 0:
            actual = cipher(text = example, shift = shift)
            expected = result
            assert actual == expected, "cipher did not accept negative shift values."

    def test_ooa(self, example, shift, result):
        """Testing if out-of-alphabet values are returned as they are"""
        symbol_regex = re.compile("[^A-Za-z]+")
        if isinstance(shift, int) and symbol_regex.search(example):
            actual = cipher(text = example, shift = shift)
            expected = result
            assert actual == expected, "Out-of-alphabet values were modified."
            
    def test_valueerror_on_string_shift(self, example, shift, result):
        """Test if submitting a string to shift argument returns TypeError."""
        if isinstance(shift, str):
            with pytest.raises(TypeError) as exception_info:
                cipher(text = example, shift = shift)
    

### Please see `test_cipher_class.py` script, and screenshot below of its command line output.

<img src="test_cipher_class_results.JPG" height = 600 width = 600/>

### 3. Use the `hypothesis` package to generate arbitrary text and integer `shift` inputs to test the `cipher` function 

In [ ]:
@given(text = st.text(min_size = 5, max_size = 10), shift = st.integers()) 
@settings(max_examples = 200)
def test_hypothesis(text, shift):
    """Testing the cipher function on single words."""
    actual = cipher(text = text, shift = shift, encrypt = True)
    expected = cipher(text = actual, shift = shift, encrypt = False)
    assert text == expected, "cipher has failed the hypothesis test."

#### Please refer to the hypothesis.py script for the setup, and see screeshot below for the command line output. Note that 200 different sets of inputs were run.

<img src="run_hypothesis_results.JPG" height  = 600 width = 600/>